In [176]:
import joblib
from pathlib import Path
import time
import vaex
import vaex.ml
from datetime import datetime
import dask
import numpy as np

dir = Path().cwd()
csv = os.path.join(dir, 'output', 'gtfsr_combined_csv.csv')

In [55]:
start = time.time()
scats_model = joblib.load('./output/scats_model.json')

print(time.time()-start)

106.8693220615387


In [170]:
if not os.path.exists(csv+'.hdf5'):
    df = vaex.from_csv(csv, convert=True, copy_index=False, chunk_size=1000000)
    df.export(csv, shuffle=True)
df = vaex.open(csv+'.hdf5')
df.head()

#,trip_id,start_date,start_time,stop_sequence,departure,arrival,timestamp,lon,lat
0,11777.2.60-13-b12-1.22.I,2.02101e+07,18:00:00,1,0,0,2021-01-09 19:28:02,0.930786,-0.112593
1,11756.2.60-13-b12-1.22.I,2.02101e+07,18:15:00,1,0,0,2021-01-09 19:28:02,0.930786,-0.112593
2,11370.2.60-13-b12-1.23.I,2.02101e+07,19:00:00,1,0,0,2021-01-09 19:28:02,0.930786,-0.112593
3,11379.2.60-13-b12-1.19.O,2.02101e+07,18:00:00,1,0,0,2021-01-09 19:28:02,0.932315,-0.109583
4,11394.2.60-13-b12-1.19.O,2.02101e+07,18:15:00,1,0,0,2021-01-09 19:28:02,0.932315,-0.109583
5,8595.2.60-4-b12-1.5.O,2.02101e+07,18:30:00,1,0,0,2021-01-09 19:28:02,0.932315,-0.109583
6,8566.2.60-4-b12-1.5.O,2.02101e+07,18:45:00,1,0,0,2021-01-09 19:28:02,0.932315,-0.109583
7,11401.2.60-13-b12-1.19.O,2.02101e+07,18:45:00,1,0,0,2021-01-09 19:28:02,0.932315,-0.109583
8,11662.2.60-27B-b12-1.97.I,2.02101e+07,19:00:00,1,0,0,2021-01-09 19:28:02,0.932315,-0.109583
9,8924.2.60-4-b12-1.5.O,2.02101e+07,19:00:00,1,0,0,2021-01-09 19:28:02,0.932315,-0.109583


In [179]:
df['dow'] = np.datetime_as_string(df.start_date).strftime('%Y%m%d').weekday()

TypeError: input must have type NumPy datetime

In [169]:
df.start_time.dtype

str

In [145]:
df.start_time = df.start_time.fillmissing('')

In [171]:
df.start_time.isna().sum()

array(0, dtype=int64)

In [175]:
# convert to dask

dask_df = df[['lon', 'lat']].to_dask_array()
dask_df

dask.array<vaex-df-0e6c5146-7044-11eb-940f, shape=(3100815, 2), dtype=float64, chunksize=(3100815, 2), chunktype=numpy.ndarray>

In [97]:
def extract_scats_from_gtfsr(df):
    # def apply_date(t):
    #     dtime = datetime.strptime(t, '%Y%m%d')
    #     dows.append(dtime.weekday())
    #     months.append(dtime.month)
    #     days.append(dtime.day)
    #     return t
    
    # def apply_time(t):
    #     dtime = datetime.strptime(t, '%H:%M:%S.%f')
    #     hours.append(dtime.hour)
    #     return t


    new_df = df
    # new_df['start_date'].apply(apply_date)
    # new_df['start_time'].apply(apply_time)
    # new_df['dow'] = [datetime.strptime(t, '%Y%m%d').weekday() for t in new_df['start_date']]
    # df['hour'] = hours
    # df['month'] = months
    # df['day'] = days

    pca_coord = vaex.ml.PCA(features=['lat', 'lon'], n_components=2, prefix='pca')
    new_df = pca_coord.fit_transform(new_df)

    # cycl_transform_hour = vaex.ml.CycleTransformer(features=['hour'], n=24)
    # new_df = cycl_transform_hour.fit_transform(new_df)

    # cycl_transform_dow = vaex.ml.CycleTransformer(features=['dow'], n=7)
    # new_df = cycl_transform_dow.fit_transform(new_df)

    # feats = new_df.get_column_names(regex='pca') + \
    #     new_df.get_column_names(regex='.*_x') + \
    #     new_df.get_column_names(regex='.*_y')

    return new_df

extract_scats_from_gtfsr(df[:5])

#,trip_id,start_date,start_time,stop_sequence,departure,arrival,timestamp,lon,lat,pca0,pca1
0,11777.2.60-13-b12-1.22.I,2.02101e+07,18:00:00,1,0,0,2021-01-09 19:28:02,0.930786,-0.112593,0.00135039,5.26479e-14
1,11756.2.60-13-b12-1.22.I,2.02101e+07,18:15:00,1,0,0,2021-01-09 19:28:02,0.930786,-0.112593,0.00135039,5.26479e-14
2,11370.2.60-13-b12-1.23.I,2.02101e+07,19:00:00,1,0,0,2021-01-09 19:28:02,0.930786,-0.112593,0.00135039,5.26479e-14
3,11379.2.60-13-b12-1.19.O,2.02101e+07,18:00:00,1,0,0,2021-01-09 19:28:02,0.932315,-0.109583,-0.00202558,-7.87896e-14
4,11394.2.60-13-b12-1.19.O,2.02101e+07,18:15:00,1,0,0,2021-01-09 19:28:02,0.932315,-0.109583,-0.00202558,-7.87896e-14


In [110]:
# dows = []
# days = []
# hours = []
# months = []

# def apply_date(t):
#     dtime = datetime.strptime(t, '%Y%m%d')
#     dows.append(dtime.weekday())
#     months.append(dtime.month)
#     days.append(dtime.day)
#     return t

# def apply_time(t):
#     dtime = datetime.strptime(t, '%H:%M:%S.%f')
#     hours.append(dtime.hour)
#     return t

# print(dows)

# df['start_date'].apply(apply_date)
# df['start_time'].apply(apply_time)
# df['dow'] = dows
# df['hour'] = hours
# df['month'] = months
# df['day'] = days
